# AllSides sources & bias crawler

Get and save a list of rated news sources as left or right and in between.

A CSV file will be created with the following columns:

- Source
- Label
- Agree
- Disagree
- Publisher URL
- Publisher site

In [1]:
!ipython -m pip install aiohttp bs4 requests

]0;IPython: research/crawlersRequirement already satisfied: aiohttp in /Users/cmin/anaconda3/lib/python3.7/site-packages (3.6.2)


In [2]:
import asyncio
import csv
import logging
import re
import urllib.parse as urlparse

import aiohttp

import bs4
import requests

In [3]:
url_tpl = "https://www.allsides.com/media-bias/media-bias-ratings?field_featured_bias_rating_value=All&field_news_source_type_tid%5B1%5D=1&field_news_source_type_tid%5B2%5D=2&field_news_source_type_tid%5B3%5D=3&field_news_bias_nid_1%5B1%5D=1&field_news_bias_nid_1%5B2%5D=2&field_news_bias_nid_1%5B3%5D=3&title=&customFilter=1&page={}"
html_parser = "html5lib"
csv_header = [
    "source",
    "label",
    "agree",
    "disagree",
    "publisher",
    "site",
]
dump_path = "media-bias.csv"
encoding = "utf-8"
skip_blocked_sites = True

verbose = True  # make it True to see debugging messages
level = logging.DEBUG if verbose else logging.INFO
logging.root.handlers.clear()
logging.basicConfig(
    format="%(levelname)s - %(name)s - %(asctime)s - %(message)s",
    level=level
)

site_adapter = {
    "www.huffingtonpost.com": "www.huffpost.com",
    "www.cnn.com": "edition.cnn.com",
    "online.wsj.com": "wsj.com",
}


async def get_soup(session, url):
    abs_url = urlparse.urljoin(url_tpl, url)
    text = await (await session.get(abs_url)).text()
#     resp.raise_for_status()
    soup = bs4.BeautifulSoup(text, html_parser)
    return soup


def _adapt_site(url, netloc):
    site = site_adapter.get(netloc)
    if site:
        url = url.replace(netloc, site)
        netloc = site
    return url, netloc


async def get_publisher_url(session, src_url, source_name):
#     import code; code.interact(local={**globals(), **locals()})
    logging.debug("Getting publisher's URL for %r.", source_name)
    soup = await get_soup(session, src_url)
    div = soup.find("div", class_="dynamic-grid")
    if not div:
        return None
    
    url = div.find("a").get("href").strip()
    parsed = urlparse.urlparse(url)
    if not parsed.netloc:
        return None
    
    return _adapt_site(url, parsed.netloc)


async def save_pages(bias_writer, csvfile):
    async with aiohttp.ClientSession() as session:
        page = 0  # custom page if you want
        while True:
            logging.info("Crawling page %d...", page)
            url = url_tpl.format(page)
            soup = await get_soup(session, url)
            
            pub_coros = []
            extras = []
            table = soup.find("table")
            if not table or "no record" in table.find("tbody").find("tr").text.lower():
                logging.info("Reached empty table -> end of results/pages.")
                break
                
            for row in table.find("tbody").find_all("tr"):
                src_a = row.find("td", class_="source-title").find("a")
                src_url = src_a.get("href")
                source_name = src_a.text
                label_alt = row.find("td", class_="views-field-field-bias-image").find("img").get("alt")
                label = label_alt.split(":")[-1].strip()
                feedback = row.find("td", class_="community-feedback")
                agree = int(feedback.find("span", class_="agree").text)
                disagree = int(feedback.find("span", class_="disagree").text)
                
                extras.append([source_name, label, agree, disagree])
#                 import code; code.interact(local={**globals(), **locals()})
                pub_coros.append(get_publisher_url(session, src_url, source_name))
            
            publisher_details_list = await asyncio.gather(*pub_coros)
            for idx, publisher_details in enumerate(publisher_details_list):
                if not publisher_details:
                    if skip_blocked_sites:
                        continue
                    else:
                        publisher_details = ("", "")

#                 print(source_name, label, f"{agree}/{disagree}")
                bias_writer.writerow(extras[idx] + list(publisher_details))

            page += 1
            csvfile.flush()


async def main():
    with open(dump_path, "w", newline="", encoding=encoding) as csvfile:
        bias_writer = csv.writer(csvfile)
        bias_writer.writerow(csv_header)
        await save_pages(bias_writer, csvfile)
        
        
await main()

INFO - root - 2020-03-27 21:27:52,466 - Crawling page 0...
DEBUG - root - 2020-03-27 21:27:54,490 - Getting publisher's URL for 'AARP'.
DEBUG - root - 2020-03-27 21:27:54,491 - Getting publisher's URL for 'ABC News'.
DEBUG - root - 2020-03-27 21:27:54,492 - Getting publisher's URL for 'Abridge News'.
DEBUG - root - 2020-03-27 21:27:54,524 - Getting publisher's URL for 'Accuracy in Media '.
DEBUG - root - 2020-03-27 21:27:54,527 - Getting publisher's URL for 'ACLU'.
DEBUG - root - 2020-03-27 21:27:54,529 - Getting publisher's URL for 'AJ+'.
DEBUG - root - 2020-03-27 21:27:54,534 - Getting publisher's URL for 'Al Cardenas'.
DEBUG - root - 2020-03-27 21:27:54,539 - Getting publisher's URL for 'Al Jazeera'.
DEBUG - root - 2020-03-27 21:27:54,545 - Getting publisher's URL for 'AllSides'.
DEBUG - root - 2020-03-27 21:27:54,550 - Getting publisher's URL for 'AllSides Community'.
DEBUG - root - 2020-03-27 21:27:54,553 - Getting publisher's URL for 'Allysia Finley (Wall Street Journal)'.
DEBUG 

INFO - root - 2020-03-27 21:28:12,302 - Crawling page 3...
DEBUG - root - 2020-03-27 21:28:14,449 - Getting publisher's URL for 'Chris Britt (cartoonist)'.
DEBUG - root - 2020-03-27 21:28:14,450 - Getting publisher's URL for 'Chris Ruddy'.
DEBUG - root - 2020-03-27 21:28:14,453 - Getting publisher's URL for 'Christian Science Monitor'.
DEBUG - root - 2020-03-27 21:28:14,455 - Getting publisher's URL for 'Christiane Amanpour'.
DEBUG - root - 2020-03-27 21:28:14,457 - Getting publisher's URL for 'Christianity Today'.
DEBUG - root - 2020-03-27 21:28:14,462 - Getting publisher's URL for 'Christopher Buskirk'.
DEBUG - root - 2020-03-27 21:28:14,484 - Getting publisher's URL for 'Chuck Asay (cartoonist)'.
DEBUG - root - 2020-03-27 21:28:14,497 - Getting publisher's URL for 'City Journal'.
DEBUG - root - 2020-03-27 21:28:14,500 - Getting publisher's URL for 'CivilPolitics.org'.
DEBUG - root - 2020-03-27 21:28:14,502 - Getting publisher's URL for 'Clay Bennett (cartoonist)'.
DEBUG - root - 202

DEBUG - root - 2020-03-27 21:28:35,253 - Getting publisher's URL for 'Forbes'.
DEBUG - root - 2020-03-27 21:28:35,262 - Getting publisher's URL for 'Foreign Affairs'.
DEBUG - root - 2020-03-27 21:28:35,273 - Getting publisher's URL for 'Foreign Policy'.
DEBUG - root - 2020-03-27 21:28:35,277 - Getting publisher's URL for 'Fox News Latino'.
DEBUG - root - 2020-03-27 21:28:35,287 - Getting publisher's URL for 'Fox News Opinion'.
DEBUG - root - 2020-03-27 21:28:35,290 - Getting publisher's URL for 'Fox Online News'.
DEBUG - root - 2020-03-27 21:28:35,293 - Getting publisher's URL for 'Frank Bruni'.
DEBUG - root - 2020-03-27 21:28:35,296 - Getting publisher's URL for 'Frank Miniter'.
DEBUG - root - 2020-03-27 21:28:35,302 - Getting publisher's URL for 'Freakonomics'.
DEBUG - root - 2020-03-27 21:28:35,305 - Getting publisher's URL for 'Freedom House'.
DEBUG - root - 2020-03-27 21:28:35,308 - Getting publisher's URL for 'Froma Harrop'.
DEBUG - root - 2020-03-27 21:28:35,310 - Getting publis

DEBUG - root - 2020-03-27 21:28:52,569 - Getting publisher's URL for 'Jonathan Miller'.
DEBUG - root - 2020-03-27 21:28:52,570 - Getting publisher's URL for "Journalist's Resource".
DEBUG - root - 2020-03-27 21:28:52,573 - Getting publisher's URL for 'Juan Williams'.
DEBUG - root - 2020-03-27 21:28:52,575 - Getting publisher's URL for 'Jubilee Media'.
DEBUG - root - 2020-03-27 21:28:52,578 - Getting publisher's URL for 'Judicial Watch'.
DEBUG - root - 2020-03-27 21:28:52,581 - Getting publisher's URL for 'Judy Sheindlin'.
DEBUG - root - 2020-03-27 21:28:52,584 - Getting publisher's URL for 'Julian Zelizer'.
DEBUG - root - 2020-03-27 21:28:52,588 - Getting publisher's URL for 'Kathleen Parker'.
DEBUG - root - 2020-03-27 21:28:52,589 - Getting publisher's URL for 'Katie Hill'.
DEBUG - root - 2020-03-27 21:28:52,591 - Getting publisher's URL for 'KATU'.
DEBUG - root - 2020-03-27 21:28:52,593 - Getting publisher's URL for 'Ken Catalino (cartoonist)'.
DEBUG - root - 2020-03-27 21:28:52,595 

DEBUG - root - 2020-03-27 21:29:10,201 - Getting publisher's URL for 'New York Magazine'.
DEBUG - root - 2020-03-27 21:29:10,202 - Getting publisher's URL for 'New York Post'.
DEBUG - root - 2020-03-27 21:29:10,204 - Getting publisher's URL for 'New York Times - News'.
DEBUG - root - 2020-03-27 21:29:10,206 - Getting publisher's URL for 'New York Times - Opinion'.
DEBUG - root - 2020-03-27 21:29:10,208 - Getting publisher's URL for 'NewsBusters'.
DEBUG - root - 2020-03-27 21:29:10,213 - Getting publisher's URL for 'Newsmax - News'.
DEBUG - root - 2020-03-27 21:29:10,216 - Getting publisher's URL for 'Newsmax - Opinion'.
DEBUG - root - 2020-03-27 21:29:10,218 - Getting publisher's URL for 'NewsOne'.
DEBUG - root - 2020-03-27 21:29:10,220 - Getting publisher's URL for 'Newsweek'.
DEBUG - root - 2020-03-27 21:29:10,223 - Getting publisher's URL for 'Newt Gingrich'.
DEBUG - root - 2020-03-27 21:29:10,228 - Getting publisher's URL for 'Nicholas Kristof'.
DEBUG - root - 2020-03-27 21:29:10,2

DEBUG - root - 2020-03-27 21:29:33,537 - Getting publisher's URL for 'Ryan Cooper'.
DEBUG - root - 2020-03-27 21:29:33,561 - Getting publisher's URL for 'S.E. Cupp'.
DEBUG - root - 2020-03-27 21:29:33,569 - Getting publisher's URL for 'Saagar Enjeti'.
DEBUG - root - 2020-03-27 21:29:33,577 - Getting publisher's URL for 'Sally Pipes'.
DEBUG - root - 2020-03-27 21:29:33,585 - Getting publisher's URL for 'Salon'.
DEBUG - root - 2020-03-27 21:29:33,591 - Getting publisher's URL for 'San Francisco Chronicle'.
DEBUG - root - 2020-03-27 21:29:33,594 - Getting publisher's URL for 'San Jose Mercury News'.
DEBUG - root - 2020-03-27 21:29:33,598 - Getting publisher's URL for 'SBG'.
DEBUG - root - 2020-03-27 21:29:33,602 - Getting publisher's URL for 'Science Daily'.
DEBUG - root - 2020-03-27 21:29:33,604 - Getting publisher's URL for 'Scientific American'.
DEBUG - root - 2020-03-27 21:29:33,607 - Getting publisher's URL for 'Scott Adams'.
DEBUG - root - 2020-03-27 21:29:33,609 - Getting publisher

DEBUG - root - 2020-03-27 21:29:55,155 - Getting publisher's URL for 'The Saturday Evening Post'.
DEBUG - root - 2020-03-27 21:29:55,157 - Getting publisher's URL for 'The Telegraph - UK'.
DEBUG - root - 2020-03-27 21:29:55,161 - Getting publisher's URL for 'The Texan'.
DEBUG - root - 2020-03-27 21:29:55,164 - Getting publisher's URL for 'The Texas Observer'.
DEBUG - root - 2020-03-27 21:29:55,166 - Getting publisher's URL for 'The Texas Tribune'.
DEBUG - root - 2020-03-27 21:29:55,168 - Getting publisher's URL for 'The Verge'.
DEBUG - root - 2020-03-27 21:29:55,176 - Getting publisher's URL for 'The Week - News'.
DEBUG - root - 2020-03-27 21:29:55,180 - Getting publisher's URL for 'The Week - Opinion'.
DEBUG - root - 2020-03-27 21:29:55,183 - Getting publisher's URL for 'The Weekly Standard'.
DEBUG - root - 2020-03-27 21:29:55,186 - Getting publisher's URL for 'The Western Journal'.
DEBUG - root - 2020-03-27 21:29:55,188 - Getting publisher's URL for 'TheBlaze.com'.
DEBUG - root - 202

Some publishers are blocked (no websites offered by AllSides), therefore fewer results in the CSV file.

Now let's find a good way of associating a side with a website in case multiple candidates are available.

In [4]:
side_dict = {}

with open(dump_path, newline="") as stream:
    reader = csv.reader(stream)
    print(next(reader))
    
    for row in reader:
        side_dict.setdefault(row[5], []).append((row[0], row[1], row[2]))

for site, sides in side_dict.items():
    if len(sides) > 1:
        print(site, sides)

['source', 'label', 'agree', 'disagree', 'publisher', 'site']
edition.cnn.com [('CNN (Web News)', 'Lean Left', '30684'), ('CNN - Editorial', 'Left', '11968')]
www.thedailybeast.com [('Daily Beast', 'Left', '9724'), ('Newsweek', 'Lean Left', '2255')]
www.foxnews.com [('Fox News Opinion', 'Right', '9912'), ('Fox Online News', 'Lean Right', '25751')]
www.courier-journal.com [('Louisville Courier-Journal', 'Lean Left', '198'), ('The Courier-Journal', 'Lean Left', '134')]
www.nytimes.com [('New York Times - News', 'Lean Left', '17886'), ('New York Times - Opinion', 'Left', '4414')]
www.newsmax.com [('Newsmax - News', 'Lean Right', '7025'), ('Newsmax - Opinion', 'Right', '58')]
www.npr.org [('NPR Editorial ', 'Lean Left', '2748'), ('NPR Online News', 'Center', '21193')]
theweek.com [('The Week - News', 'Center', '3210'), ('The Week - Opinion', 'Lean Left', '39')]
wsj.com [('Wall Street Journal - Editorial', 'Lean Right', '5941'), ('Wall Street Journal - News', 'Center', '13849')]
